In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from google.colab import files
uploaded = files.upload()

Saving basetable_ex2_4.csv to basetable_ex2_4.csv


In [3]:
basetable = pd.read_csv('basetable_ex2_4.csv')
basetable.head()

,target,gender_F,income_high,income_low,country_USA,country_India,country_UK,age,time_since_last_gift,time_since_first_gift,max_gift,min_gift,mean_gift,number_gift
0,0,1,0,1,0,1,0,65,530,2265,166.0,87.0,116.00,7
1,0,1,0,0,0,1,0,71,715,715,90.0,90.0,90.00,1
2,0,1,0,0,0,1,0,28,150,1806,125.0,74.0,96.00,9
3,0,1,0,1,1,0,0,52,725,2274,117.0,97.0,104.25,4
4,0,1,1,0,1,0,0,82,805,805,80.0,80.0,80.00,1


## Exploring the base table

1. The basetable is loaded in a pandas object basetable. Assign the number of rows to the variable population_size and print it.
2. Assign the number of targets equal to one to the variable targets_count and print it.
3. Print the target incidence, this is the ratio of targets_count and population_size.

In [4]:
# Assign the number of rows in the basetable to the variable 'population_size'.
population_size  = len(basetable)

# Print the population size.
print(population_size)

# Assign the number of targets to the variable 'targets_count'.
targets_count = sum(basetable["target"])

# Print the number of targets.
print(targets_count)

# Print the incidence, i.e. the number of targets divided by the population size.
print(targets_count/population_size)

25000
1187
0.04748


## Exploring the predictive variables

1. Count and print the number of females.
2. Count and print the number of males.

In [6]:
# Count and print the number of females.
print(sum(basetable["gender_F"]==1))

# Count and print the number of males.
print(sum(basetable["gender_F"]==0))

12579
12421


## Building a logistic regression model

1. Import the methodlinear_model from sklearn.
2. Construct a dataframe X that contains the predictors age, gender_F and time_since_last_gift.
3. Construct a dataframe y that contains the target.
4. Create a logistic regression model.
5. Fit the logistic regression model on the given basetable.

In [7]:
# Import linear_model from sklearn.
from sklearn import linear_model

# Create a dataframe X that only contains the candidate predictors age, gender_F and time_since_last_gift.
X = basetable[["age","gender_F","time_since_last_gift"]]

# Create a dataframe y that contains the target.
y = basetable[["target"]]

# Create a logistic regression model logreg and fit it to the data.
logreg = linear_model.LogisticRegression()
logreg.fit(X, y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

## Showing the coefficients and intercept

1. Assign the coefficients of the logistic regression model to the list coef.
2. Assign the intercept of the logistic regression model to the variable intercept.

In [8]:
# Construct a logistic regression model that predicts the target using age, gender_F and time_since_last gift
predictors = ["age","gender_F","time_since_last_gift"]
X = basetable[predictors]
y = basetable[["target"]]
logreg = linear_model.LogisticRegression()
logreg.fit(X, y)

# Assign the coefficients to a list coef
coef = logreg.coef_
for p,c in zip(predictors,list(coef[0])):
    print(p + '\t' + str(c))
    
# Assign the intercept to the variable intercept
intercept = logreg.intercept_
print(intercept)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


age	0.007801469599031558
gender_F	0.10964341264667059
time_since_last_gift	-0.0012872607039948116
[-2.59072469]


## Making predictions

1. Create a data frame new_data that selects the relevant columns from current data.
2. Assign to predictions the predictions for the observations in new_data.

In [17]:
current_data = basetable.loc[(len(basetable)/2):,:]
current_data.shape

(12500, 14)

In [24]:
# Fit a logistic regression model
from sklearn import linear_model
X = basetable[["age","gender_F","time_since_last_gift"]]
y = basetable[["target"]]
logreg = linear_model.LogisticRegression()
logreg.fit(X, y)

# Create a dataframe new_data from current_data that has only the relevant predictors 
new_data = current_data[["age","gender_F","time_since_last_gift"]]

# Make a prediction for each observation in new_data and assign it to predictions
predictions = logreg.predict_proba(new_data)
print(predictions[0:5])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[0.96013536 0.03986464]
 [0.94026248 0.05973752]
 [0.96865498 0.03134502]
 [0.94754252 0.05245748]
 [0.87969569 0.12030431]]


## Donor that is most likely to donate

1. The predictions are in a pandas dataframe predictions that has two columns: the donor ID and the probability to be target. Sort these predictions such that the donors with lowest probability to donate are first.
2. Select and print the row in this sorted dataframe that has the donor that is most likely to donate more than 50 Euro according to the model.

In [26]:
predictions = pd.DataFrame(predictions, columns = ['donor_ID', 'probability'])
predictions.head()

,donor_ID,probability
0,0.960135,0.039865
1,0.940262,0.059738
2,0.968655,0.031345
3,0.947543,0.052457
4,0.879696,0.120304


In [28]:
# Sort the predictions
predictions_sorted = predictions.sort_values(["probability"])

# Select the last row of the sorted predictions
print(predictions_sorted.tail(1))

      donor_ID  probability
6373  0.850223     0.149777
